# PreProcess Twitter Data

In [0]:
!pip install tweet-preprocessor
import pandas as pd
import numpy as np
import scipy as sp
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from copy import deepcopy
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer 

import preprocessor as p

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
train = pd.read_csv("/content/drive/My Drive/WebProject/labeledTrainData.csv", delimiter="\t",names=["tweet", "label", "labelValue"])
train["label"].replace({"spam": "neutral", "normal": "neutral"}, inplace=True)
train.head()

In [0]:
def expand_hashtags(tweet):
    ws.load()
    new_tweet = "" # output    
    tokens = tweet.split(' ')    
    for t in tokens:
        if t[0] == '#': # this is a hashtag, parse it
            hashtags_tokens = ws.segment(t)
            exp = " ".join(hashtags_tokens)
            new_tweet += exp
        else: # Just append the word
            new_tweet += t
        new_tweet += " "

    return new_tweet 


In [0]:
def preprocess_tweet(tweet):

    # 1. Remove URLs, emojis, mentions, smileys using tweet-preprocessor library
    # should not remove hashtags as hashtags contains important tweet content.
    
    #p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION,p.OPT.RESERVED,	p.OPT.SMILEY,p.OPT.NUMBER)
    clean = p.clean(tweet)
    # 2. Remove HTML tags using Beautiful soup library
    no_tag = BeautifulSoup(clean).get_text()

    # 3. Expanding hashtags.
    
    #tweets_expanded = expand_hashtags(no_tag)

    # 4. Remove non letter char using re
    letters_only = re.sub("[^a-zA-Z]"," ", no_tag)
    #letters_only = re.sub("[^a-zA-Z]"," ", tweets_expanded)
    
    # 5. Convert to lower case
    lower_case = letters_only.lower()
    words = lower_case.split()

    new_words = []
    ps = PorterStemmer() 
    for w in words:
        new_words.append(ps.stem(w))

    # 5. Remove stop words
    stops = set(stopwords.words("english")) 
    final_words = [w for w in new_words if not w in stops]

    return(" ".join( final_words ))

In [0]:
cleaned_tweet = []

num_tweets = train["tweet"].size
# print(num_tweets)
for i in range(num_tweets):
    cleaned_tweet.append(preprocess_tweet(train["tweet"][i]))

### **Downloading preprocessed data**

In [0]:
 df_data = pd.DataFrame(cleaned_tweet, columns=['tweet'] )
 df_data['label'] = train['label']
 df_data = df_data.reindex(columns=['tweet','label'])
 df_data.to_csv('preprocessed_tweets.csv', index = False)

# print(df_data)

## Ngram

In [0]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import nltk, re, string, collections
import operator

In [0]:
#sentiments
train_sentiment = train["label"].values
# print(cleaned_tweet[0])
# print(train_sentiment)
abusive = []
hateful = []
for x in range (train_sentiment.size):
  if train_sentiment[x] == 'abusive':
    abusive.append(cleaned_tweet[x])
  else: 
    if train_sentiment[x] == 'hateful':
      hateful.append(cleaned_tweet[x])

len(abusive)
len(hateful)

In [0]:
def create_ngrams(tweets, n_size):
  x = []
  c = Counter()
  for t in tweets:
    tokenized = t.split()
    esBigrams = ngrams(tokenized, n_size)
    esBigramFreq = collections.Counter(esBigrams)
    c = c + esBigramFreq

  d = dict(c)
  sorted_d = sorted(d.items(), key=operator.itemgetter(1), reverse=True)
  print(type(sorted_d))
  return sorted_d  
  

uni_abusive = create_ngrams(abusive,1)
bi_abusive =  create_ngrams(abusive,2)

uni_hateful = create_ngrams(hateful,1)
bi_hateful =  create_ngrams(hateful,2)

In [0]:
print(len(uni_abusive))
print(uni_abusive[:10])

print(len(bi_abusive))
print(bi_abusive[:10])

print(len(uni_hateful))
print(uni_hateful[:10])

print(len(bi_hateful))
print(bi_hateful[:10])